# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [80]:
# Your code goes here
import pandas as pd
from sklearn.impute import SimpleImputer


# Load datasets
df1 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv")
df2 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv")
df3 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv")

# Inspect datasets
print(df1.head())
print(df2.head())
print(df3.head())

# Define a cleaning function
def clean_data(df):

    df.columns = [col.lower().replace(" ", "_").replace("%", "") for col in df.columns]
    df.rename(columns = {'st':'state'},inplace=True)
    df["gender"] = df["gender"].map({'M': 'Male', 'F': 'Female', 'Femal': 'Female', 'female': 'Female', 'Male': 'Male'})
    df["state"]=df["state"].map({'AZ': 'Arizona','Cali': 'California','WA':'Washington','Washington':'Washington','Arizona':'Arizona','Nevada':'Nevada','Oregon':'Oregon','California':'California'}) 
    df["education"]=df["education"].map({'Master': 'Master','Bachelor': 'Bachelor','High School or Below':'High School or Below','College':'College','Bachelors':'Bachelor','Doctor':'Doctor'}) 

    if df['customer_lifetime_value'].dtype == 'object' and df['customer_lifetime_value'].str.contains('%').any():
        df['customer_lifetime_value'] = df['customer_lifetime_value'].str.replace('%', '').astype(float)

    df["vehicle_class"]=df["vehicle_class"].map({'Four-Door Car':'Four-Door Car','Two-Door Car':'Two-Door Car','SUV':'SUV','Sports Car': 'Luxury','Luxury SUV': 'Luxury','Luxury Car': 'Luxury'}) 

    imp = SimpleImputer(strategy='mean')
    df['customer_lifetime_value'] = imp.fit_transform(df[['customer_lifetime_value']])

    # Handle missing values and remove duplicates
    df = df.drop_duplicates()
    df = df.fillna(method='ffill').dropna()

    # Handle missing values
    df = df.dropna()
    
    # Standardize column names
    df.columns = df.columns.str.lower().str.replace(' ', '_')
    
   
    return df

# Clean individual dataframes
df1_clean = clean_data(df1)
df2_clean = clean_data(df2)
df3_clean = clean_data(df3)

# Combine dataframes
combined_df = pd.concat([df1_clean, df2_clean, df3_clean], ignore_index=True)

# Validate cleaned data
cleaned_missing_values = missing_value_analysis(combined_df)
print("Cleaned Data Missing Values:")
print(cleaned_missing_values)

# Save the cleaned combined dataframe
combined_df.to_csv("cleaned_combined_data.csv", index=False)

print("Data cleaned and combined successfully.")


  Customer          ST GENDER             Education Customer Lifetime Value  \
0  RB50392  Washington    NaN                Master                     NaN   
1  QZ44356     Arizona      F              Bachelor              697953.59%   
2  AI49188      Nevada      F              Bachelor             1288743.17%   
3  WW63253  California      M              Bachelor              764586.18%   
4  GA49547  Washington      M  High School or Below              536307.65%   

    Income  Monthly Premium Auto Number of Open Complaints     Policy Type  \
0      0.0                1000.0                    1/0/00   Personal Auto   
1      0.0                  94.0                    1/0/00   Personal Auto   
2  48767.0                 108.0                    1/0/00   Personal Auto   
3      0.0                 106.0                    1/0/00  Corporate Auto   
4  36357.0                  68.0                    1/0/00   Personal Auto   

   Vehicle Class  Total Claim Amount  
0  Four-Door Car 

C:\Users\Admin\AppData\Local\Temp\ipykernel_20928\1244274377.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()
C:\Users\Admin\AppData\Local\Temp\ipykernel_20928\1244274377.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()
C:\Users\Admin\AppData\Local\Temp\ipykernel_20928\1244274377.py:35: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill').dropna()


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [71]:
import pandas as pd
import numpy as np

# 1. EDA Functions
def summary_statistics(df, columns=None):
    if columns:
        return df[columns].describe()
    return df.describe()

def missing_value_analysis(df):
    missing = df.isnull().sum()
    missing_percent = 100 * df.isnull().sum() / len(df)
    return pd.DataFrame({'Missing Values': missing, 'Percentage': missing_percent})

def distribution_analysis(df, column):
    return df[column].value_counts(normalize=True)

def correlation_analysis(df, method='pearson'):
    return df.corr(method=method)

def outlier_detection(df, column, method='iqr'):



    if method == 'iqr':
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]
    elif method == 'zscore':
        from scipy.stats import zscore
        outliers = df[(np.abs(zscore(df[column].dropna())) > 3)]
    else:
        raise ValueError("Method should be 'iqr' or 'zscore'")
    return outliers

# 2. Data Cleaning Functions
def remove_null_values(df, columns=None):
    if columns:
        return df.dropna(subset=columns)
    return df.dropna()

def remove_duplicates(df):
    return df.drop_duplicates()

def missing_value_imputation(df, columns, strategy='mean'):
    for column in columns:
        if strategy == 'mean':
            df[column].fillna(df[column].mean(), inplace=True)
        elif strategy == 'median':
            df[column].fillna(df[column].median(), inplace=True)
        elif strategy == 'mode':
            df[column].fillna(df[column].mode()[0], inplace=True)
        else:
            raise ValueError("Strategy should be 'mean', 'median', or 'mode'")
    return df

def treat_missing_values(df, strategy='mean'):
    for column in df.columns:
        if df[column].dtype in ['int64', 'float64']:
            if strategy == 'mean':
                df[column].fillna(df[column].mean(), inplace=True)
            elif strategy == 'median':
                df[column].fillna(df[column].median(), inplace=True)
            elif strategy == 'mode':
                df[column].fillna(df[column].mode()[0], inplace=True)
    return df

# 3. Data Formatting Functions
def normalize_data(df, column):
    df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())
    return df

def one_hot_encode(df, column):
    return pd.get_dummies(df, columns=[column], drop_first=True)

def label_encode(df, column):
    from sklearn.preprocessing import LabelEncoder
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    return df

def extract_date_features(df, column):
    df[column] = pd.to_datetime(df[column])
    df['year'] = df[column].dt.year
    df['month'] = df[column].dt.month
    df['day'] = df[column].dt.day
    df['weekday'] = df[column].dt.weekday
    df['hour'] = df[column].dt.hour
    return df

def remove_duplicates_and_treat_nulls(df, null_strategy='mean'):
    df = remove_duplicates(df)
    df = treat_missing_values(df, strategy=null_strategy)
    return df

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

# Initial Exploration (EDA)
summary_stats = summary_statistics(df)
print("Summary Statistics:")
print(summary_stats)

missing_values = missing_value_analysis(df)
print("\nMissing Values:")
print(missing_values)

# Data Cleaning
df = remove_duplicates(df)
df = treat_missing_values(df, strategy='mean')

# Data Formatting
print("\nCleaned and Formatted DataFrame:")
print(df.head())

df.to_csv('cleaned_marketing_customer_analysis.csv', index=False)

Summary Statistics:
         unnamed:_0  customer_lifetime_value        income  \
count  10910.000000             10910.000000  10910.000000   
mean    5454.500000              8018.241094  37536.284785   
std     3149.590053              6885.081434  30359.195670   
min        0.000000              1898.007675      0.000000   
25%     2727.250000              4014.453113      0.000000   
50%     5454.500000              5771.147235  33813.500000   
75%     8181.750000              8992.779137  62250.750000   
max    10909.000000             83325.381190  99981.000000   

       monthly_premium_auto  months_since_last_claim  \
count          10910.000000             10910.000000   
mean              93.196059                15.149071   
std               34.442532                 9.783520   
min               61.000000                 0.000000   
25%               68.000000                 7.000000   
50%               83.000000                15.000000   
75%              109.000000  

In [62]:
# Your code goes here
%load_ext autoreload
%autoreload 2

# Load the dataset
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv"
df = pd.read_csv(url)

# 1. Initial Exploration (EDA)

# Summary statistics
summary_stats = summary_statistics(df)
print("Summary Statistics:")
print(summary_stats)

# Missing value analysis
missing_values = missing_value_analysis(df)
print("\nMissing Values:")
print(missing_values)

# 2. Data Cleaning

# Remove duplicates
df = remove_duplicates(df)

# Treat missing values for numeric columns (using mean as an example)
df = treat_missing_values(df, strategy='mean')


# 3. Data Formatting

# Final cleaned and formatted DataFrame
print("\nCleaned and Formatted DataFrame:")
print(df.head())

# Save the cleaned and formatted dataframe to a new CSV file (optional)
df.to_csv('cleaned_marketing_customer_analysis.csv', index=False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Summary Statistics:
         unnamed:_0  customer_lifetime_value        income  \
count  10910.000000             10910.000000  10910.000000   
mean    5454.500000              8018.241094  37536.284785   
std     3149.590053              6885.081434  30359.195670   
min        0.000000              1898.007675      0.000000   
25%     2727.250000              4014.453113      0.000000   
50%     5454.500000              5771.147235  33813.500000   
75%     8181.750000              8992.779137  62250.750000   
max    10909.000000             83325.381190  99981.000000   

       monthly_premium_auto  months_since_last_claim  \
count          10910.000000             10910.000000   
mean              93.196059                15.149071   
std               34.442532                 9.783520   
min               61.000000                 0.000000   
25%               68.000000                 7.000000 

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [63]:
# Convert 'effective_to_date' to datetime format if not already converted
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

In [47]:
# Create a pivot table to summarize total revenue for each sales channel
pivot_table = df.pivot_table(values='total_claim_amount', 
                             index='sales_channel', 
                             aggfunc='sum').round(2)

# Sort the pivot table to see which sales channel has the most revenue
pivot_table = pivot_table.sort_values(by='total_claim_amount', ascending=False)

# Display the pivot table
print(pivot_table)

### Insights
print(f"After creating the pivot table for total revenue by sales channel, it was observed that the Agent had the highest revenue, followed by the Branch.")

               total_claim_amount
sales_channel                    
Agent                  1810226.82
Branch                 1301204.00
Call Center             926600.82
Web                     706600.04


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [82]:
# Create pivot table to analyze customer lifetime value
pivot_table = df.pivot_table(values='customer_lifetime_value', index='gender', columns='education', aggfunc='mean')

# Display the pivot table
display(pivot_table)
### Insights
print("It appears males generally have higher customer lifetime value across most education levels.")


education,Bachelor,College,Doctor,High School or Below,Master
gender,,,,,
F,7874.269478,7748.823325,7328.508916,8675.220201,8157.053154
M,7703.601675,8052.459288,7415.333638,8149.687783,8168.832659


It appears males generally have higher customer lifetime value across most education levels.


## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [87]:
# Ensure correct format and extract month
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])
df['month'] = df['effective_to_date'].dt.month
# Group and summarize complaints
complaints_summary = df.groupby(['policy_type', 'month'])['number_of_open_complaints'].sum().reset_index()
complaints_summary.columns = ['Policy Type', 'Month', 'Number of Complaints']
complaints_summary['Month'] = pd.Categorical(complaints_summary['Month'], categories=range(1, 13), ordered=True)
complaints_summary = complaints_summary.sort_values('Month')
# Display summary
print(complaints_summary)
### Insights
print("We can observe that except for Special Auto, the higuest numbers of complaints were registred in the month of january.")

      Policy Type Month  Number of Complaints
0  Corporate Auto     1            443.434952
2   Personal Auto     1           1727.605722
4    Special Auto     1             87.074049
1  Corporate Auto     2            385.208135
3   Personal Auto     2           1453.684441
5    Special Auto     2             95.226817
We can observe that except for Special Auto, the higuest numbers of complaints were registred in the month of january.
